# Partie B 

In [1]:
#!pip install earthengine-api
import ee
ee.Authenticate()
ee.Initialize()


Enter verification code:  4/1ASc3gC1jjYUrn8iYZEQsV_lM2LHN3nEsk40T9yislPQrt6V7n7vYZTrtvEY



Successfully saved authorization token.


In [19]:
import ee
import folium
import os

# Initialisation Earth Engine
ee.Initialize()

# Créer le dossier output
output_dir = "output"
os.makedirs(output_dir, exist_ok=True)

# Définir la zone d'intérêt : Ghana
countries = ee.FeatureCollection('FAO/GAUL/2015/level0')
ghana = countries.filter(ee.Filter.eq('ADM0_NAME', 'Ghana'))

# Daily total precipitation sums (m)
era5 = (
    ee.ImageCollection('ECMWF/ERA5/DAILY')
    .select('total_precipitation')
    .filter(ee.Filter.date('2015-01-01', '2020-12-31'))
)

# Filtrer sur le Ghana
data = era5.filterBounds(ghana)

# Fonction pour calculer la moyenne annuelle
def annual_mean(year):
    start = ee.Date.fromYMD(year, 1, 1)
    end = start.advance(1, 'year')
    mean_img = data.filterDate(start, end).mean()
    return mean_img.set('year', year)

# Liste des années
years = list(range(2015, 2021))

# Créer une collection d'images annuelles
annual_means = ee.ImageCollection([annual_mean(y) for y in years])

# Exemple : afficher la moyenne pour 2015
mean_2015 = annual_means.filter(ee.Filter.eq('year', 2015)).first()

# Convertir en mm/an 
mean_2015_mm = mean_2015.multiply(1000)  
print(mean_2015_mm)
# Visualisation sur une carte Folium
map_center = ghana.geometry().centroid().coordinates().getInfo()[::-1]
m = folium.Map(location=map_center, zoom_start=6)

# Palette de couleurs
vis_params = {
    'min': 0,
    'max': 20, 
    'palette': ['#f7fbff', '#6baed6', '#08306b']
}
# Fonction pour ajouter un raster EE sur Folium
def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Google Earth Engine',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

# Ajouter la fonction à Folium
folium.Map.add_ee_layer = add_ee_layer

# Ajouter la couche à la carte
m.add_ee_layer(mean_2015_mm, vis_params, 'Précipitations moyennes 2015 (mm/jour)')

# Afficher la carte
m.save(os.path.join(output_dir, "ghana_precip_2015.html"))
print(f"Carte sauvegardée dans {output_dir}/ghana_precip_2015.html")
# Exporter les moyennes annuelles en GeoTIFF
for y in years:
    img = annual_means.filter(ee.Filter.eq('year', y)).first().multiply(1000)  # mm/jour
    task = ee.batch.Export.image.toDrive(
        image=img.clip(ghana),
        description=f'ghana_precip_mean_{y}',
        folder='GEE_exports',
        fileNamePrefix=f'ghana_precip_mean_{y}',
        region=ghana.geometry(),
        scale=10000,
        crs='EPSG:4326',
        maxPixels=1e13
    )
    task.start()
    print(f"Tâche d'export lancée pour {y}")

# Fonction pour ajouter des couches EE à Folium
def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data © Google Earth Engine',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

# Ajouter la méthode à Folium
folium.Map.add_ee_layer = add_ee_layer

# Exporter les moyennes annuelles en GeoTIFF
for y in years:
    img = annual_means.filter(ee.Filter.eq('year', y)).first().multiply(1000)  # mm/jour
    task = ee.batch.Export.image.toDrive(
        image=img.clip(ghana),
        description=f'ghana_precip_mean_{y}',
        folder='GEE_exports',
        fileNamePrefix=f'ghana_precip_mean_{y}',
        region=ghana.geometry(),
        scale=10000,
        crs='EPSG:4326',
        maxPixels=1e13
    )
    task.start()
    print(f"Tâche d'export lancée pour {y}")

# Fonction pour ajouter des couches EE à Folium
def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data © Google Earth Engine',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

# Ajouter la méthode à Folium
folium.Map.add_ee_layer = add_ee_layer


ee.Image({
  "functionInvocationValue": {
    "functionName": "Image.multiply",
    "arguments": {
      "image1": {
        "functionInvocationValue": {
          "functionName": "Collection.first",
          "arguments": {
            "collection": {
              "functionInvocationValue": {
                "functionName": "Collection.filter",
                "arguments": {
                  "collection": {
                    "functionInvocationValue": {
                      "functionName": "ImageCollection.fromImages",
                      "arguments": {
                        "images": {
                          "arrayValue": {
                            "values": [
                              {
                                "functionInvocationValue": {
                                  "functionName": "Element.set",
                                  "arguments": {
                                    "key": {
                                      "constantValue": "year"
